In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

In [7]:
md = pd. read_csv('/Users/claireyuen/Desktop/Movie-Recommendation-System/archive/movies_metadata.csv')
md.head()

/Users/claireyuen/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


## Demographic Filtering
It offers a generalized recommendation to evey user based on the popularity and genre. The idea of this filtering is that the movies are more popular will have a higher probability to be liked by the general audiences. 

Weighted Rating formula is used to costruct a fair Top Movie Chart with average number of votes and high ratings. 
<br>v: The number of votes for the movie
<br>m: The minimum votes required to be listed in the chart
<br>R: The average rating of the movie
<br>C: The mean vote across the whole report
<br>For the value of m, 95th percentile is the cutoff point. It must have more votes than at least 95% of the movies in the list.

In [8]:
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] 
                                                                   if isinstance(x, list) else [])

In [9]:
counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
C = averages.mean()
C

5.244896612406511

In [10]:
#95th percentile
m = counts.quantile(0.95)
m

434.0

In [116]:
md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] 
                                                                       if x != np.nan else np.nan)
qualify = md[(md['vote_count'] >= m) & (md['vote_count'].notnull()) 
               & (md['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]
qualify['vote_count'] = qualified['vote_count'].astype('int')
qualify['vote_average'] = qualified['vote_average'].astype('int')
qualify.shape

(2274, 6)

To be qualify to Top Movie Chart, a movie must have a minimum votes of 434, and average rating in 5.2449 on the scale of 10. There are 2274 movies that qualified to the Top Movie Chart. 

In [117]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [118]:
qualify['weighted_rating'] = qualify.apply(weighted_rating, axis=1)
qualify = qualify.sort_values('weighted_rating', ascending=False).head(100)

### Top 10 Movies

In [119]:
qualify.head(10)

,title,year,vote_count,vote_average,popularity,genres,weighted_rating
15480,Inception,2010,14075,8,29.108149,"[Action, Thriller, Science Fiction, Mystery, A...",7.917588
12481,The Dark Knight,2008,12269,8,123.167259,"[Drama, Action, Crime, Thriller]",7.905871
22879,Interstellar,2014,11187,8,32.213481,"[Adventure, Drama, Science Fiction]",7.897107
2843,Fight Club,1999,9678,8,63.869599,[Drama],7.881753
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.070725,"[Adventure, Fantasy, Action]",7.871787
292,Pulp Fiction,1994,8670,8,140.950236,"[Thriller, Crime]",7.868660
314,The Shawshank Redemption,1994,8358,8,51.645403,"[Drama, Crime]",7.864000
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.324358,"[Adventure, Fantasy, Action]",7.861927
351,Forrest Gump,1994,8147,8,48.307194,"[Comedy, Drama, Romance]",7.860656
5814,The Lord of the Rings: The Two Towers,2002,7641,8,29.423537,"[Adventure, Fantasy, Action]",7.851924


### Top Movies by Genres

In [120]:
s = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_md = md.drop('genres', axis=1).join(s)

/var/folders/jq/04f89pcj5sq7z15gzmgc3q780000gn/T/ipykernel_12996/2744528521.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)


In [121]:
def chart_genre(genre, percentile=0.85):
    df1 = gen_md[gen_md['genre'] == genre]
    counts = df1[df1['vote_count'].notnull()]['vote_count'].astype('int')
    averages = df1[df1['vote_average'].notnull()]['vote_average'].astype('int')
    C = averages.mean()
    m = counts.quantile(percentile)
    
    qualify = df1[(df1['vote_count'] >= m) & (df1['vote_count'].notnull()) & 
                 (df1['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    qualify['vote_count'] = qualify['vote_count'].astype('int')
    qualify['vote_average'] = qualify['vote_average'].astype('int')
    
    qualify['weighted_rating'] = qualify.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) 
                                  + (m/(m+x['vote_count']) * C), axis=1)
    qualify = qualify.sort_values('weighted_rating', ascending=False).head(110)
    
    return qualify

In [122]:
chart_genre('Romance').head(10)

,title,year,vote_count,vote_average,popularity,weighted_rating
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457024,8.565285
351,Forrest Gump,1994,8147,8,48.307194,7.971357
876,Vertigo,1958,1162,8,18.20822,7.811667
40251,Your Name.,2016,1030,8,34.461252,7.789489
883,Some Like It Hot,1959,835,8,11.845107,7.745154
1132,Cinema Paradiso,1988,834,8,14.177005,7.744878
19901,Paperman,2012,734,8,7.198633,7.713951
37863,Sing Street,2016,669,8,10.672862,7.689483
882,The Apartment,1960,498,8,11.994281,7.599317
38718,The Handmaiden,2016,453,8,16.727405,7.566166


## Content Based Filtering
This filtering analyzes the contents (storyline, genre, cast, director etc.) of the movie to find out other movies which have similar content. It ranks similar movies according to their similarity scores and recommends the most relevant movies to the user

In [123]:
links_small = pd.read_csv('/Users/claireyuen/Desktop/Movie-Recommendation-System/archive/links_small.csv')
links_small.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [124]:
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [125]:
md = md.drop([19730, 29503, 35587])

In [126]:
md['id'] = md['id'].astype('int')

In [127]:
smd = md[md['id'].isin(links_small)]
smd.shape

(9099, 25)

In [128]:
smd.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995


### Movie Description Based Recommender

In [129]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

/var/folders/jq/04f89pcj5sq7z15gzmgc3q780000gn/T/ipykernel_12996/3912728496.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['tagline'] = smd['tagline'].fillna('')
/var/folders/jq/04f89pcj5sq7z15gzmgc3q780000gn/T/ipykernel_12996/3912728496.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['description'] = smd['overview'] + smd['tagline']
/var/folders/jq/04f89pcj5sq7z15gzmgc3q780000gn/T/ipykernel_12996/3912728496.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [130]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

In [131]:
tfidf_matrix.shape

(9099, 268124)

Cosine Similarity is used to calculate the quantity of similar denotes between two movies. 

In [132]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [133]:
smd = smd.reset_index()
titles = smd['title']
index = pd.Series(smd.index, index=smd['title'])

Write a function that returns the 30 most similar movies based on the cosine similarity score.

In [189]:
#Recommender based on Movie Overviews and Taglines
def recommendations(title):
    idx = index[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_index = [i[0] for i in sim_scores]
    return titles.iloc[movie_index]

In [137]:
recommendations('Toy Story 2').head(10)

0                             Toy Story
7535                        Toy Story 3
8413               Toy Story of Terror!
6193             The 40 Year Old Virgin
1480                     Small Soldiers
1777                               Toys
2547                    Man on the Moon
8914                       Search Party
6188                     The Great Raid
7420    Rudolph, the Red-Nosed Reindeer
Name: title, dtype: object

The above recommender doesn't take into considerations very important features such as cast, crew, director and genre, which determine the rating and the popularity of a movie.

### Metadata Based Recommender
The following recommender will take genre, keywords, cast and crew into consideration.
(genres, director, main actors and keywords)

In [138]:
credits = pd.read_csv('/Users/claireyuen/Desktop/Movie-Recommendation-System/archive/credits.csv')
keywords = pd.read_csv('/Users/claireyuen/Desktop/Movie-Recommendation-System/archive/keywords.csv')

In [139]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')

In [140]:
md = md.merge(credits, on='id')
md = md.merge(keywords, on='id')

In [141]:
smd = md[md['id'].isin(links_small)]
smd.shape

(9219, 28)

In [142]:
smd.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,status,tagline,title,video,vote_average,vote_count,year,cast,crew,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,Released,NaN,Toy Story,False,7.7,5415.0,1995,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


#### Wrangling the dataframe:
Crew: From the crew, we will only pick the director as our feature
<br>Cast: Choose the top 3 actors that appear in the credits list.

In [143]:
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))

/var/folders/jq/04f89pcj5sq7z15gzmgc3q780000gn/T/ipykernel_12996/3047342568.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['cast'] = smd['cast'].apply(literal_eval)
/var/folders/jq/04f89pcj5sq7z15gzmgc3q780000gn/T/ipykernel_12996/3047342568.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['crew'] = smd['crew'].apply(literal_eval)
/var/folders/jq/04f89pcj5sq7z15gzmgc3q780000gn/T/ipykernel_12996/3047342568.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

In [144]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [145]:
smd['director'] = smd['crew'].apply(get_director)

/var/folders/jq/04f89pcj5sq7z15gzmgc3q780000gn/T/ipykernel_12996/4176871582.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['director'] = smd['crew'].apply(get_director)


In [146]:
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

/var/folders/jq/04f89pcj5sq7z15gzmgc3q780000gn/T/ipykernel_12996/864329085.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
/var/folders/jq/04f89pcj5sq7z15gzmgc3q780000gn/T/ipykernel_12996/864329085.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)


In [147]:
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

/var/folders/jq/04f89pcj5sq7z15gzmgc3q780000gn/T/ipykernel_12996/4209080348.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])


### Count Vectorizer (like Description Recommender):
1. Strip Spaces and Convert to Lowercase
2. Mention Director 3 times to give it more weight relative to the entire cast

In [148]:
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

/var/folders/jq/04f89pcj5sq7z15gzmgc3q780000gn/T/ipykernel_12996/997285517.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])


In [149]:
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
smd['director'] = smd['director'].apply(lambda x: [x,x, x])

/var/folders/jq/04f89pcj5sq7z15gzmgc3q780000gn/T/ipykernel_12996/1272965743.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
/var/folders/jq/04f89pcj5sq7z15gzmgc3q780000gn/T/ipykernel_12996/1272965743.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['director'] = smd['director'].apply(lambda x: [x,x, x])


### Keywords
Calculate the frequenct counts of every keyword that appears in the dataset.
<br>Keywords occur in frequencies ranging from 1 to 610. We do not have any use for keywords that occur only once. Therefore, these can be safely removed. Finally, we will convert every word to its stem so that words such as Dogs and Dog are considered the same.

In [150]:
s = smd.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'
s = s.value_counts()

/var/folders/jq/04f89pcj5sq7z15gzmgc3q780000gn/T/ipykernel_12996/49438598.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = smd.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)


In [153]:
s = s[s > 1]
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

'dog'

In [154]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [155]:
smd['keywords'] = smd['keywords'].apply(filter_keywords)
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

/var/folders/jq/04f89pcj5sq7z15gzmgc3q780000gn/T/ipykernel_12996/1613546145.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['keywords'] = smd['keywords'].apply(filter_keywords)
/var/folders/jq/04f89pcj5sq7z15gzmgc3q780000gn/T/ipykernel_12996/1613546145.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
/var/folders/jq/04f89pcj5sq7z15gzmgc3q780000gn/T/ipykernel_12996/1613546145.py:3: SettingWithCopyWarning: 
A value is t

In [156]:
smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))

/var/folders/jq/04f89pcj5sq7z15gzmgc3q780000gn/T/ipykernel_12996/1585552675.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']
/var/folders/jq/04f89pcj5sq7z15gzmgc3q780000gn/T/ipykernel_12996/1585552675.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))


In [158]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [159]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [190]:
#Recommender based on Movie Cast, Crew, Keywords and Genre
def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year']]
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

In [161]:
improved_recommendations('Toy Story 2')

/var/folders/jq/04f89pcj5sq7z15gzmgc3q780000gn/T/ipykernel_12996/2657751039.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualified['vote_count'] = qualified['vote_count'].astype('int')
/var/folders/jq/04f89pcj5sq7z15gzmgc3q780000gn/T/ipykernel_12996/2657751039.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualified['vote_average'] = qualified['vote_average'].astype('int')
/var/folders/jq/04f89pcj5sq7z15gzmgc3q780000gn/T/ipykernel_12996/2657751039.py:16: SettingWithCopyWarning: 
A value is try

,title,vote_count,vote_average,year,wr
7307,Up,7048,7,2009,6.898194
0,Toy Story,5415,7,1995,6.869770
8432,Despicable Me 2,4729,7,2013,6.852467
7629,Toy Story 3,4710,7,2010,6.851922
8595,The Lego Movie,3127,7,2014,6.786095
6496,Cars,3991,6,2006,5.925940
1883,A Bug's Life,2379,6,1998,5.883500
7334,Ice Age: Dawn of the Dinosaurs,2330,6,2009,5.881435
7404,Cloudy with a Chance of Meatballs,1799,6,2009,5.853240
7914,Cars 2,2088,5,2011,5.042143


## Collaborative Filtering
The recommender captures the personal tastes and biases of a user. It based on the idea that users similar to a me can be used to predict how much I will like a particular product or service those users have used/experienced but I have not.

In [162]:
ratings = pd.read_csv('/Users/claireyuen/Desktop/Movie-Recommendation-System/archive/ratings_small.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [177]:
reader = Reader()
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
svd = SVD()
cross_validate(svd, data, measures=['rmse', 'mae'], cv=5,verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8972  0.8959  0.8922  0.9020  0.8940  0.8963  0.0033  
MAE (testset)     0.6909  0.6892  0.6876  0.6938  0.6871  0.6897  0.0024  
Fit time          7.32    8.10    9.31    7.59    7.62    7.99    0.71    
Test time         0.24    0.22    1.33    0.21    0.21    0.44    0.44    


{'test_rmse': array([0.89718512, 0.89591872, 0.89218279, 0.90202294, 0.89399261]),
 'test_mae': array([0.69085774, 0.6891948 , 0.6876059 , 0.69376359, 0.68705461]),
 'fit_time': (7.317737102508545,
  8.096922874450684,
  9.306602001190186,
  7.594870090484619,
  7.620826959609985),
 'test_time': (0.23943495750427246,
  0.21751809120178223,
  1.3254508972167969,
  0.209367036819458,
  0.2055809497833252)}

Root Mean Sqaure Error(RMSE): 0.89

In [174]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [175]:
ratings[ratings['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [179]:
svd.predict(1, 300, 3, verbose=True)

user: 1          item: 300        r_ui = 3.00   est = 2.83   {'was_impossible': False}


Prediction(uid=1, iid=300, r_ui=3, est=2.8319826101971017, details={'was_impossible': False})

For movie ID 300, the estimated prediction is 2.83.

## Hybird Recommendation
Combine content-based and collaborative filtering
<br> Input: User ID and the Title of a Movie
<br> Output: Similar movies sorted on the basis of expected ratings by that particular user

In [180]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [181]:
id_map = pd.read_csv('/Users/claireyuen/Desktop/Movie-Recommendation-System/archive/links_small.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(smd[['title', 'id']], on='id').set_index('title')

In [182]:
indices_map = id_map.set_index('id')

In [183]:
def hybrid(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    #print(idx)
    movie_id = id_map.loc[title]['movieId']
    
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [187]:
hybrid(2, 'Toy Story')

,title,vote_count,vote_average,year,id,est
7629,Toy Story 3,4710.0,7.6,2010,10193,3.947196
3833,"Monsters, Inc.",6150.0,7.5,2001,585,3.927194
2522,Toy Story 2,3914.0,7.3,1999,863,3.889787
7404,Cloudy with a Chance of Meatballs,1799.0,6.5,2009,22794,3.868991
7169,A Matter of Loaf and Death,120.0,7.2,2008,14447,3.868854
4341,"The Looney, Looney, Looney Bugs Bunny Movie",13.0,6.6,1981,41394,3.783636
2751,Creature Comforts,29.0,7.3,1989,54825,3.649329
8519,Toy Story of Terror!,246.0,7.3,2013,213121,3.625415
3016,Chicken Run,1190.0,6.5,2000,7443,3.612962
6386,Luxo Jr.,148.0,7.1,1986,13925,3.584372


In [188]:
hybrid(400, 'Toy Story')

,title,vote_count,vote_average,year,id,est
7629,Toy Story 3,4710.0,7.6,2010,10193,4.155238
3833,"Monsters, Inc.",6150.0,7.5,2001,585,4.120891
8595,The Lego Movie,3127.0,7.5,2014,137106,3.983919
2751,Creature Comforts,29.0,7.3,1989,54825,3.888295
2522,Toy Story 2,3914.0,7.3,1999,863,3.876562
4341,"The Looney, Looney, Looney Bugs Bunny Movie",13.0,6.6,1981,41394,3.829966
7404,Cloudy with a Chance of Meatballs,1799.0,6.5,2009,22794,3.829830
7169,A Matter of Loaf and Death,120.0,7.2,2008,14447,3.768137
5287,"Bon Voyage, Charlie Brown (and Don't Come Back!)",12.0,6.2,1980,31112,3.694581
8519,Toy Story of Terror!,246.0,7.3,2013,213121,3.683865


Different recommendations is made for different users although the movie is the same. Hence, our recommendations are more personalized and tailored towards particular users.